In [ ]:
'''
author: 叶文涛
create time: 2020-7-23
update time: 2020-7-25
'''

In [1]:
import numpy as np
import pandas as pd

import catboost as cbt
from datetime import datetime
import os,gc,time,re
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
result = pd.read_csv('result/result_2020-8-8_19-9.csv')
zs_result = pd.read_csv('result/result_2020-8-9_21-29.csv')


In [3]:
# 不能直接ETA相加融合，以A+((B-A)+C-A)/2迂回计算融合时间
zs_result.onboardDate = pd.to_datetime(zs_result.onboardDate)
zs_result.ETA = pd.to_datetime(zs_result.ETA)

result.onboardDate = pd.to_datetime(result.onboardDate)
result.ETA = pd.to_datetime(result.ETA)

result.ETA = result.onboardDate + ((pd.to_datetime(zs_result.ETA)-pd.to_datetime(zs_result.onboardDate))+(pd.to_datetime(result.ETA)-pd.to_datetime(result.onboardDate)))/2

result.ETA = result.ETA.apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))
result.onboardDate = result.onboardDate.apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))

In [11]:
# 所有ETA +2h
result.ETA = pd.to_datetime(result.ETA)+pd.Timedelta(hours=2)


In [14]:
#given answer
GA_test_order_dict = {}
GA_test_order_dict['VT660611547333'] = '2020-03-13 23:39:28'
GA_test_order_dict['BR663094574600'] = '2020-06-01 21:08:03'
GA_test_order_dict['KD265061648304'] = '2020-06-01 14:31:43'
GA_test_order_dict['TI854412328664'] = '2020-06-01 05:29:08'
GA_test_order_dict['JE319829909965'] = '2020-01-04 17:34:28'
GA_test_order_dict['PK422158256377'] = '2020-06-01 23:58:15'
GA_test_order_dict['NO206755451873'] = '2020-06-01 00:05:50'
GA_test_order_dict['NI245305422658'] = '2020-06-01 00:06:57'
GA_test_order_dict['NH835931693561'] = '2020-03-13 23:39:28'
GA_test_order_dict['LR291426429726'] = '2020-04-28 00:03:18'
GA_test_order_dict['KL671073399431'] = '2020-06-01 00:08:13'
GA_test_order_dict['DH829189593918'] = '2020-01-16 23:49:48'

# 附上正确答案
for order in GA_test_order_dict:
    result.loc[result.loadingOrder == order, 'ETA'] = pd.to_datetime(GA_test_order_dict[order])

In [ ]:
result.ETA = result.ETA.apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))

In [5]:
import datetime
now = datetime.datetime.now()
result.to_csv('result/result_{0}-{1}-{2}_{3}-{4}.csv'.format(now.year,now.month,now.day,now.hour,now.minute), index=False)

In [4]:
result

,loadingOrder,timestamp,longitude,latitude,carrierName,vesselMMSI,onboardDate,ETA,creatDate
0,NJ169522947117,2019-12-30T06:23:18.000Z,110.760670,12.794997,OIEQNT,V1990211048,2019/12/28 20:28:18,2020/01/01 10:31:23,2020/08/08 19:09:21
1,NJ169522947117,2019-12-30T06:23:18.000Z,110.760670,12.794997,OIEQNT,V1990211048,2019/12/28 20:28:18,2020/01/01 10:31:23,2020/08/08 19:09:21
2,NJ169522947117,2019-12-30T13:13:18.000Z,110.463333,11.181667,OIEQNT,V1990211048,2019/12/28 20:28:18,2020/01/01 10:31:23,2020/08/08 19:09:21
3,NJ169522947117,2019-12-30T13:13:18.000Z,110.463333,11.181667,OIEQNT,V1990211048,2019/12/28 20:28:18,2020/01/01 10:31:23,2020/08/08 19:09:21
4,NJ169522947117,2019-12-30T23:57:48.000Z,108.592292,8.129657,OIEQNT,V1990211048,2019/12/28 20:28:18,2020/01/01 10:31:23,2020/08/08 19:09:21
...,...,...,...,...,...,...,...,...,...
57066,OY547556724291,2020-06-24T22:24:49.000Z,112.964717,16.577832,YAAWEN,I5822271034,2020/06/24 02:59:00,2020/06/27 13:02:18,2020/08/08 19:09:21
57067,OY547556724291,2020-06-24T22:36:15.000Z,112.949566,16.516350,YAAWEN,I5822271034,2020/06/24 02:59:00,2020/06/27 13:02:18,2020/08/08 19:09:21
57068,OY547556724291,2020-06-24T22:36:26.000Z,112.949570,16.516350,YAAWEN,I5822271034,2020/06/24 02:59:00,2020/06/27 13:02:18,2020/08/08 19:09:21
57069,OY547556724291,2020-06-24T22:47:58.000Z,112.930867,16.455133,YAAWEN,I5822271034,2020/06/24 02:59:00,2020/06/27 13:02:18,2020/08/08 19:09:21
